## Test problem

As a test example, we will model a clamped beam deformed under its own weight in 3D. This can be modeled by setting the right-hand side body force per unit volume to  
**f = (0, 0, –ρg)**  
with **ρ** the density of the beam and **g** the acceleration of gravity.

The beam is box-shaped with length **L** and has a square cross section of width **W**.  
We set **u = u_D = (0, 0, 0)** at the clamped end, **x = 0**.  
The rest of the boundary is traction-free, that is, we set **T = 0**.

We start by defining the physical variables used in the program.


In [2]:
# Scaled variable
import pyvista
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
L = 1
W = 0.2
mu = 1
rho = 1
delta = W / L
gamma = 0.4 * delta**2
beta = 1.25
lambda_ = beta
g = gamma

We then create the mesh, which will consist of hexahedral elements, along with the function space.

As we want a vector element with three components, we add **(3, )** or **(domain.geometry.dim, )** to the element tuple to make it a triplet.

However, we also could have used `basix.ufl`'s functionality, creating a vector element:

```python
element = basix.ufl.element("Lagrange", domain.topology.cell_name(), 1, shape=(domain.geometry.dim,))
V = dolfinx.fem.functionspace(domain, element)


In [3]:
domain = mesh.create_box(MPI.COMM_WORLD, [np.array([0, 0, 0]), np.array([L, W, W])],
                         [20, 6, 6], cell_type=mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, )))


## Boundary conditions

As we would like to clamp the boundary at **x = 0**, we do this by using a marker function, which locates the facets where **x** is close to zero by machine precision.


In [4]:
def clamped_boundary(x):
    return np.isclose(x[0], 0)


fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)

u_D = np.array([0, 0, 0], dtype=default_scalar_type)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V)

As we want the traction **T** over the remaining boundary to be 0, we create a `dolfinx.Constant`.


In [5]:
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

We also want to specify the integration measure **ds**, which should be the integral over the boundary of our domain.

We do this by using `ufl`, and its built-in integration measures.


In [6]:
ds = ufl.Measure("ds", domain=domain)

## Variational formulation

We are now ready to create our variational formulation in close-to-mathematical syntax, as for the previous problems.


In [7]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))  # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)


def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(domain, default_scalar_type((0, 0, -rho * g)))
a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds

## Solve the linear variational problem

As in the previous demos, we assemble the matrix and right-hand side vector and use PETSc to solve our variational problem.


In [8]:
problem = LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

## Deformation Visualization in ParaView

> **Note:** I am still working out this part of the workflow. I chose to use ParaView for visualization due to compatibility issues with `pyvista` on my system.

To visualize the computed deformation, I save the solution using an `XDMFFile`. After saving, the file can be opened in **ParaView**.

Open `deformation.xdmf` in ParaView, press **Apply**, and then click the **Warp by Vector** button  
(or navigate via `Filters → Alphabetical → Warp by Vector`) and press **Apply** again.

To color the deformed beam, use the dropdown in the **color menu** and change the value from `Solid Color` to `Deformation`.


In [9]:
with io.XDMFFile(domain.comm, "deformation.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    uh.name = "Deformation"
    xdmf.write_function(uh)

## Stress computation

As soon as the displacement is computed, we can compute various stress measures.  
We will compute the von Mises stress defined as:

**σ_m = √(3/2) · s : s**

where **s** is the deviatoric stress tensor:

**s(u) = σ(u) – (1/3) tr(σ(u)) I**


In [10]:
s = sigma(uh) - 1. / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh))
von_Mises = ufl.sqrt(3. / 2 * ufl.inner(s, s))

The `von_Mises` variable is now an expression that must be projected into an appropriate function space so that we can visualize it.

As `uh` is a linear combination of first-order piecewise continuous functions, the von Mises stresses will be a cell-wise constant function.


In [11]:
V_von_mises = fem.functionspace(domain, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
stresses = fem.Function(V_von_mises)
stresses.interpolate(stress_expr)

In the previous sections, we have only visualized first-order Lagrangian functions.

However, the von Mises stresses are piecewise constant on each cell.  
Therefore, we modify our plotting routine slightly.

The first thing we notice is that we now set values for each cell, which has a one-to-one correspondence with the degrees of freedom in the function space.


In [13]:
from dolfinx import io

with io.XDMFFile(domain.comm, "stresses.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    stresses.name = "VonMises"
    xdmf.write_function(stresses)
